# PD 2
## Hubert Ruczyński

# Importy

In [1]:
%matplotlib inline
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import sklearn
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
np.random.seed(23)
import category_encoders as ce

# ***Część I***

## Treść

Wykonaj target encoding dla zmiennej it_location. Czy i jakie są przewagi target encoding nad one-hot? Jako target traktujemy kolumnę price (będzie to więc zadanie regresji).

Zastosuj trzy metody encodingu (one-hot + "dwie nowe") dla kolumny main_category. "Nowe metody" proszę wybrać spośród wymienionych na stronie https://contrib.scikit-learn.org/category_encoders/. W przypadku, gdy użyta metoda nie działa proszę o stosowną adnotację. Opisz wyniki.

Zwizalizuj wynik oraz wyjaśnij czym się różnią sposoby kodowania (czemu to działa).

In [2]:
transactions_df = pd.read_csv('allegro-api-transactions.csv', index_col='lp')
transactions_df.sample(3)

,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
lp,,,,,,,,,,,,,
303208,2016-04-03 18:17:20,6068256052,"['Delikatesy', 'Herbaty', 'Czarne']",1,0,ik-mercado-uk,22.9,0,19,0,5295,Lincoln,Delikatesy
130798,2016-04-03 23:18:29,5948515466,"['Dom i Ogród', 'Wyposażenie', 'Oświetlenie', ...",1,0,toptools_com_pl,25.0,0,0,0,1549,Białystok,Dom i Ogród
257614,2016-04-03 19:53:27,6059197530,"['Odzież, Obuwie, Dodatki', 'Obuwie', 'Męskie'...",1,1,N1A_PL,249.0,1,0,0,537,Szczecin,"Odzież, Obuwie, Dodatki"


In [3]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420020 entries, 0 to 420019
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date                    420020 non-null  object 
 1   item_id                 420020 non-null  int64  
 2   categories              420020 non-null  object 
 3   pay_option_on_delivery  420020 non-null  int64  
 4   pay_option_transfer     420020 non-null  int64  
 5   seller                  420020 non-null  object 
 6   price                   420020 non-null  float64
 7   it_is_allegro_standard  420020 non-null  int64  
 8   it_quantity             420020 non-null  int64  
 9   it_is_brand_zone        420020 non-null  int64  
 10  it_seller_rating        420020 non-null  int64  
 11  it_location             420020 non-null  object 
 12  main_category           420020 non-null  object 
dtypes: float64(1), int64(7), object(5)
memory usage: 44.9+ MB


In [4]:
len(transactions_df.loc[:,'it_location'].unique())

10056

In [5]:
transactions_df.head()

,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
lp,,,,,,,,,,,,,
0,2016-04-03 21:21:08,4753602474,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,59.99,1,997,0,50177,Warszawa,Komputery
1,2016-04-03 15:35:26,4773181874,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,4.90,1,9288,0,12428,Warszawa,"Odzież, Obuwie, Dodatki"
2,2016-04-03 14:14:31,4781627074,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,109.90,1,895,0,7389,Leszno,Dom i Ogród
3,2016-04-03 19:55:44,4783971474,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",1,1,Matfel1,18.50,0,971,0,15006,Wola Krzysztoporska,Książki i Komiksy
4,2016-04-03 18:05:54,4787908274,"['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",1,1,PPHU_RICO,19.90,1,950,0,32975,BIAŁYSTOK,"Odzież, Obuwie, Dodatki"


In [6]:
print(transactions_df.isnull().sum())

date                      0
item_id                   0
categories                0
pay_option_on_delivery    0
pay_option_transfer       0
seller                    0
price                     0
it_is_allegro_standard    0
it_quantity               0
it_is_brand_zone          0
it_seller_rating          0
it_location               0
main_category             0
dtype: int64


In [7]:
X=transactions_df.drop('price', axis =1)
y=transactions_df.drop('it_location', axis=1)

## Target Encoder

In [8]:
encoder = ce.TargetEncoder(cols=['it_location'])
encoder.fit(X,y['price'])
df_target_encoder=encoder.transform(X,y['price'])
df_target_encoder.head(3)

,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,it_is_allegro_standard,it_quantity,it_is_brand_zone,it_seller_rating,it_location,main_category
lp,,,,,,,,,,,,
0,2016-04-03 21:21:08,4753602474,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,1,997,0,50177,85.423398,Komputery
1,2016-04-03 15:35:26,4773181874,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,1,9288,0,12428,85.423398,"Odzież, Obuwie, Dodatki"
2,2016-04-03 14:14:31,4781627074,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,1,895,0,7389,61.990914,Dom i Ogród


In [9]:
len(df_target_encoder.loc[:,'it_location'].unique())

6283

In [10]:
onehot = ce.OneHotEncoder(handle_unknown='ignore')
encoded_onehot = pd.DataFrame(onehot.fit_transform(transactions_df[["main_category"]]))
data_onehot = transactions_df.join(encoded_onehot)
data_onehot.head(3)

,date,item_id,categories,pay_option_on_delivery,pay_option_transfer,seller,price,it_is_allegro_standard,it_quantity,it_is_brand_zone,...,main_category_18,main_category_19,main_category_20,main_category_21,main_category_22,main_category_23,main_category_24,main_category_25,main_category_26,main_category_27
lp,,,,,,,,,,,,,,,,,,,,,
0,2016-04-03 21:21:08,4753602474,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",1,1,radzioch666,59.99,1,997,0,...,0,0,0,0,0,0,0,0,0,0
1,2016-04-03 15:35:26,4773181874,"['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",1,1,InwestycjeNET,4.90,1,9288,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-04-03 14:14:31,4781627074,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",1,1,otostyl_com,109.90,1,895,0,...,0,0,0,0,0,0,0,0,0,0


Inspirując się artykułem: https://medium.com/analytics-vidhya/target-encoding-vs-one-hot-encoding-with-simple-examples-276a7e7b3e64, udało mi się wyróżnić proste wady i zalety każdego z kodowań.

**OneHot**

*Zalety:*

Działa dobrze w przypadku danych nominalnych i eliminuje problem z wyższymi wartościami kategorycznymi wpływającymi na dane, ponieważ tworzymy każdą kolumnę w binarnej 1 lub 0.

*Wady:*

Może stworzyć bardzo dużą wymiarowość w zależności od liczby posiadanych cech kategorycznych i liczby kategorii na obiekt, co w ostateczności doprowadzić może do bardzo działania.

**Target**

*Opis:*

Polega na tym, że funkcje są zastępowane połączeniem późniejszego prawdopodobieństwa celu przy danej określonej wartości kategorycznej i wcześniejszego prawdopodobieństwa celu względem wszystkich danych uczących.

*Zalety:*

Prosta i szybka metoda kodowania, która nie zwiększa wymiarowości zbioru danych.

*Wady:*

Zależy ono od dystrybucji celu, co oznacza, że wymaga starannej weryfikacji, ponieważ może być podatne na overfitting. Ta metoda jest również specyficzna dla zbioru danych i będzie wykazywać znaczną poprawę tylko przez pewien czas.

## ***OneHotEncoding***

In [11]:
X3=transactions_df.drop('price', axis =1)
y3=transactions_df.drop('main_category', axis =1)
ce_one_hot = ce.OneHotEncoder(cols = ['main_category'])
ce_one_hot_df=ce_one_hot.fit_transform(X3,y3)

In [12]:
ce_one_hot_df.iloc[:, 11:38]

,main_category_1,main_category_2,main_category_3,main_category_4,main_category_5,main_category_6,main_category_7,main_category_8,main_category_9,main_category_10,...,main_category_18,main_category_19,main_category_20,main_category_21,main_category_22,main_category_23,main_category_24,main_category_25,main_category_26,main_category_27
lp,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420015,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
420016,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
420017,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
OneHot=ce_one_hot_df.iloc[:, 11:38]
OneHot=OneHot.sum()
OneHot

main_category_1     14491
main_category_2     54257
main_category_3     91042
main_category_4     11572
main_category_5      5808
main_category_6     20341
main_category_7     45941
main_category_8     42107
main_category_9     28096
main_category_10    19805
main_category_11    27532
main_category_12     8074
main_category_13     6146
main_category_14     7150
main_category_15     6574
main_category_16    13166
main_category_17     5959
main_category_18     1005
main_category_19     2381
main_category_20     3194
main_category_21      617
main_category_22     1961
main_category_23     1053
main_category_24      413
main_category_25     1214
main_category_26      119
main_category_27        2
dtype: int64

In [14]:
bars = ('main_category_1', 'main_category_2','main_category_3', 'main_category_4','main_category_5', 'main_category_6','main_category_7', 'main_category_8','main_category_9', 'main_category_10','main_category_11', 'main_category_12','main_category_13', 'main_category_14','main_category_15', 'main_category_16','main_category_17', 'main_category_18','main_category_19', 'main_category_20','main_category_21', 'main_category_22','main_category_23', 'main_category_24','main_category_25','main_category_26','main_category_27')
y_pos = np.arange(len(bars))
plt.bar(y_pos,OneHot)

<BarContainer object of 27 artists>

Metoda OneHotEncoding działa w ten sposób, że rozdziela zadaną kolumnę na wszystkie unikalne kategorie (kolejne kolumny zamiast tamtej) i następnie dla każdej zmiennej daje 1 w miejscu gdzie chodzi o daną kategorię, a 0 jeśli nie.

## ***LeaveOneOut***

In [15]:
X4=transactions_df.drop('price', axis =1)
y4=transactions_df.drop('main_category', axis =1)
ce_leave = ce.LeaveOneOutEncoder(cols = ['main_category'])
ce_leave.fit(X4, y4['price'])        
ce_leave_df=ce_leave.transform(X4, y4['price'])

In [16]:
leave=ce_leave_df.iloc[:, 11:38]
leave

,main_category
lp,
0,121.814331
1,75.859357
2,72.434342
3,25.028061
4,75.859080
...,...
420015,107.530709
420016,28.129044
420017,75.859337


In [17]:
sns.histplot(leave)

<AxesSubplot:ylabel='Count'>

Kodowanie Leave One Out  oblicza średnią zmiennych docelowych dla wszystkich rekordów zawierających tę samą wartość dla danej zmiennej jakościowej. Algorytm kodowania różni się nieco w zestawie danych uczących i testowych. 

W przypadku zestawu danych uczących omawiany rekord jest pomijany, stąd nazwa Leave One Out. Kodowanie jest następujące dla pewnej wartości określonej zmiennej kategorialnej.

W przypadku danych walidacyjnych lub zestawu danych prognozowania definicja jest nieco inna. Nie musimy zostawiać aktualnego rekordu i nie potrzebujemy czynnika losowości.

## ***Helmert Encoding***

In [18]:
X5=transactions_df.drop('price', axis =1)
y5=transactions_df.drop('main_category', axis =1)
ce_helmert = ce.HelmertEncoder(cols = ['main_category'])
ce_helmert_df=ce_helmert .fit_transform(X5, y5)

In [19]:
helmert=ce_helmert_df.iloc[:, 13:39]
helmert

,main_category_1,main_category_2,main_category_3,main_category_4,main_category_5,main_category_6,main_category_7,main_category_8,main_category_9,main_category_10,...,main_category_16,main_category_17,main_category_18,main_category_19,main_category_20,main_category_21,main_category_22,main_category_23,main_category_24,main_category_25
lp,,,,,,,,,,,,,,,,,,,,,
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,0.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420015,0.0,0.0,0.0,5.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
420016,0.0,0.0,0.0,0.0,0.0,0.0,8.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
420017,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [20]:
helmert.max(axis=1)

lp
0        -1.0
1        -1.0
2         2.0
3         3.0
4        -1.0
         ... 
420015    5.0
420016    8.0
420017   -1.0
420018    7.0
420019    6.0
Length: 420020, dtype: float64

In [21]:
n_bins=27
plt.hist(helmert.max(axis=1), bins=n_bins)
plt.show()

Metoda Helmerta działa w sposób podobny do do OneHotEncoding, gdyż w miejscu w któym mamy do czynienia z n-tą kategorią wstawiana jest liczba n odpowiadająca danej kategorii. Pozostałe kategorie są obsługiwane w taki sposób, że te wcześniejsze otrzymują 0 zaś późniejsze 1.

Kodowanie Helmerta jest typem kodowania kategorialnego dla regresji. Porównuje każdy poziom zmiennej kategorialnej ze średnią kolejnych poziomów. Stąd pierwszy kontrast porównuje średnią zmiennej zależnej ze średnią wszystkich kolejnych poziomów kolumny kategorialnej, drugi kontrast porównuje średnią zmienna zależna ze średnią wszystkich kolejnych poziomów, itd.

# ***Część II***

## Treść

W tej części zadania traktujemy zmienną price nie jak target a zmienną objaśniającą. Zbiór danych ograniczamy do zmiennych numerycznych tj. price, it_seller_rating i it_quantity.

Proszę losowo usunąć 10% wartości ze zmiennej it_seller_rating i je uzupełnić z użyciem jednego z automatycznych narzędzi: Nearest neighbors imputation lub Multivariate feature imputation (https://scikit-learn.org/stable/modules/impute.html).
Następnie należy porównać wartości imputowane z oryginalnymi (polecam miarę RMSE). Eksperyment powtórzyć 10 razy i zobaczyć jakie będzie odchylenie standardowe wyniku. Następnie zrobić analogiczną analizę gdy oprócz losowego usuwania 10% wartości z kolumny it_seller_rating usuniemy także losowo 10% ze zmiennej it_quantity. (w przypadku problemów wydajnościowych proszę ograniczyć liczbę rekordów).

Opisać wnioski z analizy jakości imputacji i umieścić podsumowujący wykres.

## **Pojedyncza iteracja**

Najpierw dokonajmy pojedynczej iteracji dla opisanego zadania z usuwaniem danych tylko z jednej kolumny.

In [22]:
df=transactions_df[['price', 'it_seller_rating', 'it_quantity']]
df2=df.copy()
df2

,price,it_seller_rating,it_quantity
lp,,,
0,59.99,50177,997
1,4.90,12428,9288
2,109.90,7389,895
3,18.50,15006,971
4,19.90,32975,950
...,...,...,...
420015,180.00,176,0
420016,14.99,34851,2
420017,5.99,983,470


In [23]:
df2.loc[df.sample(frac=0.1).index, 'it_seller_rating'] = np.nan
df2

,price,it_seller_rating,it_quantity
lp,,,
0,59.99,NaN,997
1,4.90,NaN,9288
2,109.90,7389.0,895
3,18.50,15006.0,971
4,19.90,32975.0,950
...,...,...,...
420015,180.00,176.0,0
420016,14.99,34851.0,2
420017,5.99,983.0,470


In [24]:
df2.isnull().sum()

price                   0
it_seller_rating    42002
it_quantity             0
dtype: int64

Do imputacji brakujących zmiennych używam IterativeImputera (Multivariate feature imputation)

In [25]:
imp = IterativeImputer(random_state=0, skip_complete=True, n_nearest_features=10)
df3=imp.fit_transform(df2)
df3=pd.DataFrame(df3, columns=['price','it_seller_rating','it_quantity'])
df3

,price,it_seller_rating,it_quantity
0,59.99,19999.720065,997.0
1,4.90,20657.103009,9288.0
2,109.90,7389.000000,895.0
3,18.50,15006.000000,971.0
4,19.90,32975.000000,950.0
...,...,...,...
420015,180.00,176.000000,0.0
420016,14.99,34851.000000,2.0
420017,5.99,983.000000,470.0
420018,200.00,163.000000,0.0


In [26]:
df3.isnull().sum()

price               0
it_seller_rating    0
it_quantity         0
dtype: int64

Imputer zadziałał, gdyż nie mamy brakujących danych.

In [27]:
sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1],squared=False)

11507.569618984044

In [28]:
np.std(df, ddof=0)

price                 390.325807
it_seller_rating    36682.854968
it_quantity         23387.220223
dtype: float64

In [29]:
np.std(df3, ddof=0)

price                 390.325807
it_seller_rating    34832.035651
it_quantity         23387.220223
dtype: float64

## 10 iteracji dla jednej kategorii wybrakowanych

In [30]:
imp = IterativeImputer(max_iter=10000, random_state=10, n_nearest_features=100,initial_strategy='mean')
v1 = []
for i in range(10):
    df2=df.copy()
    df2.loc[df.sample(frac=0.1).index, 'it_seller_rating'] = np.nan
    df3=imp.fit_transform(df2)
    df3=pd.DataFrame(df3, columns=['price','it_seller_rating','it_quantity'])
    v1.append(sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1], squared=False))
    print("Mean squared Error : " + str(sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1], squared=False)))
    print('Standard deviation for all variables:')
    print(np.std(df3, ddof=0))
    print(" ")
std1 = np.std(v1, axis=0)
print('Standard deviation for RMSE: '+str(std1))

Mean squared Error : 11401.017466338135
Standard deviation for all variables:
price                 390.325807
it_seller_rating    34865.105063
it_quantity         23387.220223
dtype: float64
 
Mean squared Error : 11677.52635351068
Standard deviation for all variables:
price                 390.325807
it_seller_rating    34775.968590
it_quantity         23387.220223
dtype: float64
 
Mean squared Error : 11612.84799626132
Standard deviation for all variables:
price                 390.325807
it_seller_rating    34795.648502
it_quantity         23387.220223
dtype: float64
 
Mean squared Error : 11639.78808142354
Standard deviation for all variables:
price                 390.325807
it_seller_rating    34788.366122
it_quantity         23387.220223
dtype: float64
 
Mean squared Error : 11629.618088613672
Standard deviation for all variables:
price                 390.325807
it_seller_rating    34790.464123
it_quantity         23387.220223
dtype: float64
 
Mean squared Error : 11571.786368

## 10 iteracji dla dwóch kategorii wybrakowanych

In [31]:
imp = IterativeImputer(max_iter=10000, random_state=10, n_nearest_features=100,initial_strategy='mean')
v2 = []
v3 = []
for i in range(10):
    df2=df.copy()
    df2.loc[df.sample(frac=0.1).index, 'it_seller_rating'] = np.nan
    df2.loc[df.sample(frac=0.1).index, 'it_quantity'] = np.nan
    df3=imp.fit_transform(df2)
    df3=pd.DataFrame(df3, columns=['price','it_seller_rating','it_quantity'])
    v2.append(sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1], squared=False))
    v3.append(sklearn.metrics.mean_squared_error(df.iloc[:, 2],df3.iloc[:, 2], squared=False))
    print("Mean squared Error it_seller_rating: " + str(sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1], squared=False)))
    print("Mean squared Error it_quantity: " + str(sklearn.metrics.mean_squared_error(df.iloc[:, 2],df3.iloc[:, 2], squared=False)))
    print('Standard deviation for all variables:')
    print(np.std(df3, ddof=0))
    print(" ")
std2 = np.std(v2, axis=0)
print('Standard deviation for it_seller_rating RMSE: '+str(std2))
std3 = np.std(v3, axis=0)
print('Standard deviation for it_quantity RMSE: '+str(std3))

Mean squared Error it_seller_rating: 11574.76685844917
Mean squared Error it_quantity: 7348.832862331095
Standard deviation for all variables:
price                 390.325807
it_seller_rating    34810.851363
it_quantity         22203.462209
dtype: float64
 
Mean squared Error it_seller_rating: 11552.92749238203
Mean squared Error it_quantity: 7298.312636911238
Standard deviation for all variables:
price                 390.325807
it_seller_rating    34819.068065
it_quantity         22221.046166
dtype: float64
 
Mean squared Error it_seller_rating: 11464.379125973754
Mean squared Error it_quantity: 7431.675859415434
Standard deviation for all variables:
price                 390.325807
it_seller_rating    34846.305560
it_quantity         22180.109755
dtype: float64
 
Mean squared Error it_seller_rating: 11751.00788517532
Mean squared Error it_quantity: 7406.151929658769
Standard deviation for all variables:
price                 390.325807
it_seller_rating    34749.133554
it_quantity  

## Wnioski po imputacji:

W obu przypadkach imputacja jest wysoce nieskuteczna, gdyż błąd średniokwadratowy jest duży.

Widoczne są też zmiany w wariancjach zmiennych które były uzupełniane.

Z drugiej strony stron, w przypadku braków danych, jest to coś z czego faktycznie opłaca się korzystać, gdyż jesteśmy w stanie pozyskać jedynie w małym,stopniu zaburzone rekordy, za to w dużej ilości.

In [32]:
itera=[1,2,3,4,5,6,7,8,9,10]
iters=[0,1,2,3,4,5,6,7,8,9]

## Wizualizacje RMSE

In [33]:
sns.set(style="whitegrid", color_codes=True)
fig, ax = plt.subplots() 
sns.barplot(itera, y=v1, color='blue', saturation=0.5)
ax.set_ylim(11000,12000)
plt.title('RMSE for first test')
plt.show()

In [34]:
sns.set(style="whitegrid", color_codes=True)
fig, ax = plt.subplots() 
sns.barplot(itera, y=v2, color='cyan')
ax.set_ylim(11000,12000)
plt.title('RMSE for second test, middle column')
plt.show() 

In [35]:
sns.set(style="whitegrid", color_codes=True)
fig, ax = plt.subplots() 
sns.barplot(itera, y=v3, color='darkblue')
ax.set_ylim(7000,7600)
plt.title('RMSE for second test, last column')
plt.show() 